In [ ]:
import numpy as np 
import ROOT
from ROOT import TMVA, TFile, TCut
from root_numpy.tmva import add_classification_events, evaluate_reader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, average_precision_score, cohen_kappa_score
from sacred import Experiment
from sacred.observers import MongoObserver
from sacred.observers import FileStorageObserver
import matplotlib.pyplot as plt

In [ ]:
#%jsmva on

In [ ]:
#Define training vars
basic_vars = ['met_met', 'met_phi',
            'lead_lep_pt', 'lead_lep_eta', 'lead_lep_phi','lead_lep_e',
             'jet_pt[0]', 'jet_eta[0]', 'jet_phi[0]', 'jet_e[0]','isbtagged_MV2c10_85[0]',
             'jet_pt[1]', 'jet_eta[1]', 'jet_phi[1]', 'jet_e[1]','isbtagged_MV2c10_85[1]']

In [ ]:
#Define full vars
train_vars = ['met_met',
            'lead_lep_pt', 'lead_lep_eta','lead_lep_e',
             'jet_pt[0]', 'jet_eta[0]', 'jet_e[0]','jet_isbtagged_MV2c10_85[0]',
             'jet_pt[1]', 'jet_eta[1]', 'jet_e[1]','jet_isbtagged_MV2c10_85[1]', 
             'HT_all', 'Centrality_all', 'dEtajl_MaxdEta', 'dRbl_MindPhi_MV2c10_85', 'dRbj_MaxdEta_MV2c10_85', 'dRjl_MindR_MV2c10_85',
             "Aplanarity_jets", 'H0_all', 'nFJets', 'nBJets_MV2c10_85', 'nJets_Pt30', 'nJets_Pt40', 'HardCentralVeto_MV2c10_85', 'H2_jets']

In [ ]:
#Define training file names
data_dir = "data/"
sgtop_file = data_dir + "sgtop_train.npy"  
ttbar_file = data_dir + "ttbar_train.npy"   
wjets_file = data_dir + "wjets_train.npy" 
sig_file   = data_dir + "sig_train.npy"  

In [ ]:
#Extract training arrays
sgtop_arr = np.load(sgtop_file, encoding="bytes")
ttbar_arr = np.load(ttbar_file, encoding="bytes")
wjets_arr = np.load(wjets_file, encoding="bytes")
sig_arr   = np.load(sig_file, encoding="bytes")

In [ ]:
#Transform into dataframes
sgtop_df = pd.DataFrame(sgtop_arr)
ttbar_df = pd.DataFrame(ttbar_arr)
wjets_df = pd.DataFrame(wjets_arr)
sig_df = pd.DataFrame(sig_arr)

In [ ]:
#Label as sig or back
sgtop_df['label'] = 0
ttbar_df['label'] = 0
wjets_df['label'] = 0
sig_df['label'] = 1

In [ ]:
del sgtop_arr
del ttbar_arr
del wjets_arr
del sig_arr

In [ ]:
#Concat arrays, define train and validation sets, transform to lgb dataset
full = pd.concat([sig_df, sgtop_df, ttbar_df, wjets_df])

train, valid = train_test_split(full, test_size=0.2)
trainlen = len(full['met_met'].values)

In [ ]:
len(sig_df)

In [ ]:
del sgtop_df
del ttbar_df
del wjets_df
del sig_df
del full

In [ ]:
len(train_vars)

In [ ]:
test.columns = [x.replace('[', '_').replace(']', '') for x in test.columns.values]

In [ ]:
from root_numpy import array2root

In [ ]:
array2root(test.to_records(), 'cuts.root', 'cuts')

In [ ]:
from root_numpy import root2array

In [ ]:
c = root2array('cuts.root', 'cuts', selection = sel)

In [ ]:
c['label']

In [ ]:
sel = "dRjl_MindR_MV2c10_85 > 2.0 && nFJets >= 1 && abs(dRbl_MindPhi_MV2c10_85) > 2.5 && jet_pt_0 > 350000 &&  jet_isbtagged_MV2c10_85_0 == 1 && HardCentralVeto_MV2c10_85 == 0 && met_met > 120000"

In [ ]:
o = root2array('tmva_output.root', './TestTree', selection=sel)

In [ ]:
o

In [ ]:
np.sum(o['classID'])/len(o)

In [ ]:
len(test)

In [ ]:
pred = np.where(o['BDT'] > 0.5, 1, 0)

In [ ]:
np.sum(pred)

In [ ]:
rtest = pd.DataFrame(c)#test[pred == 1]
s = 36100*np.sum(rtest[rtest['label'] == 1].apply(lambda x : x['weight_normalise']*x['weight_mc']*x['weight_pileup']*x['weight_jvt']*x['weight_leptonSF']*x['weight_bTagSF_MV2c10_85'], axis = 1))
b = 36100*np.sum(rtest[rtest['label'] == 0].apply(lambda x : x['weight_normalise']*x['weight_mc']*x['weight_pileup']*x['weight_jvt']*x['weight_leptonSF']*x['weight_bTagSF_MV2c10_85'], axis = 1))

In [ ]:
s/np.sqrt(b)*5/np.sqrt(5)

In [ ]:
#output = TFile('tmva_output.root', 'recreate')


In [ ]:
#factory = TMVA.Factory('classifier', output,
#                       'AnalysisType=Classification:'
#                      '!V:Silent:!DrawProgressBar')

In [ ]:
#data = TMVA.DataLoader('.')

In [ ]:
output = TFile('tmva_output.root', 'recreate')
factory = TMVA.Factory('classifier', output,
                       'AnalysisType=Classification:'
                       '!V:Silent:!DrawProgressBar')
data = TMVA.DataLoader('.')
clean_train_vars = [x.replace('[', '_').replace(']', '') for x in train_vars]
for var in clean_train_vars:
    data.AddVariable(var)

In [ ]:
#clean_train_vars = [x.replace('[', '_').replace(']', '') for x in train_vars]

In [ ]:
#for var in clean_train_vars:
#    data.AddVariable(var)

In [ ]:
add_classification_events(data, train[train_vars].values, train['label'].values)#, weights=w_train)

In [ ]:
del train

In [ ]:
sgtop_file = data_dir + "sgtop_test.npy"  
ttbar_file = data_dir + "ttbar_test.npy"   
wjets_file = data_dir + "wjets_test.npy" 
sig_file   = data_dir + "sig_test.npy"  

In [ ]:
#Extract training arrays
sgtop_arr = np.load(sgtop_file, encoding="bytes")
ttbar_arr = np.load(ttbar_file, encoding="bytes")
wjets_arr = np.load(wjets_file, encoding="bytes")
sig_arr   = np.load(sig_file, encoding="bytes")

In [ ]:
#Transform into dataframes
sgtop_df = pd.DataFrame(sgtop_arr)
ttbar_df = pd.DataFrame(ttbar_arr)
wjets_df = pd.DataFrame(wjets_arr)
sig_df = pd.DataFrame(sig_arr)

In [ ]:
#Label as sig or back
sgtop_df['label'] = 0
ttbar_df['label'] = 0
wjets_df['label'] = 0
sig_df['label'] = 1

In [ ]:
test = pd.concat([sig_df, sgtop_df, ttbar_df, wjets_df])
testlen = len(test)

In [ ]:
del sgtop_arr
del ttbar_arr
del wjets_arr
del sig_arr

In [ ]:
del sgtop_df
del ttbar_df
del wjets_df
del sig_df

In [ ]:
testlen + trainlen

In [ ]:
add_classification_events(data, test[train_vars].values, test['label'].values, test=True)#, weights=w_train)

In [ ]:
del test

In [ ]:
# The following line is necessary if events have been added individually:
data.PrepareTrainingAndTestTree(TCut('1'), 'NormMode=EqualNumEvents')


In [ ]:
#params = "!V:NTrees=200:MinNodeSize=2.5%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20"

In [ ]:
#factory.BookMethod(data, TMVA.Types.kBDT, "BDT",
#                   params );



In [ ]:
#factory.BookMethod( data, TMVA.Types.kBDT, "BDT",
#   H=False, V=False, NTrees=10, MinNodeSize="2.5%", MaxDepth=3, BoostType="AdaBoost", AdaBoostBeta=0.5,
#                  UseBaggedBoost=True, BaggedSampleFraction=0.5, SeparationType="GiniIndex", nCuts=20, SigToBkgFraction=10)


In [ ]:
factory.BookMethod( data, TMVA.Types.kBDT, "BDT",
   "!H:V:MaxDepth=5:BoostType=AdaBoost:AdaBoostBeta=0.05:PruningValFraction=0.1")

In [ ]:
%%time 
factory.TrainAllMethods()

In [ ]:
factory.TestAllMethods()


In [ ]:
factory.EvaluateAllMethods()

In [ ]:
factory.DrawROCCurve(data)

In [ ]:
#factory.DrawCutEfficiencies(data, "BDT")

In [ ]:
factory.EvaluateAllMethods()

In [ ]:


fpr, tpr, _ = roc_curve(test['label'], pred_vals)
roc_auc = roc_auc_score(test['label'], pred_vals)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
pred = np.where(pred_vals > np.mean(_), 1, 0)

In [ ]:
confusion_matrix(pred, test['label'])

In [ ]:
average_precision_score(pred, test['label'])


In [ ]:
cohen_kappa_score(pred, test['label'])

In [ ]:
roc_auc